### 1. DATA COLLECTION

In [ ]:
#pip install bs4
#pip install selenium
#pip install forex-python

In [5]:
#import the required packages
import requests
import bs4
from bs4 import BeautifulSoup
import pandas as pd
import os
import time
import json
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import heapq
import nltk
from nltk.stem import *
from nltk.corpus import stopwords # import stopwords module

#### 1.1. Get the list of master's degree courses

In [147]:
flag = False # run only once

if flag:
    # URL of the website
    url = 'https://www.findamasters.com/masters-degrees/msc-degrees/?PG='  
    prefix = '/masters-degrees/course/'
    exclude = ['\nMore details \n', '\nRead more \n', '\xa0Video(s)', '\xa0Student Profile(s)'] 

    # Create a list to store the URLs of the masters
    master_urls = []

    # Loop through the first 400 pages
    for page_number in range(1, 401):
        print(url + str(page_number))
        response = requests.get(url + str(page_number))

        if response.status_code == 200:
            soup = BeautifulSoup(response.text, "html.parser")
        
            # Use BeautifulSoup to extract the URLs and append them to the master_urls list
            for link in soup.find_all('a', {'class':'courseLink'}):
                if link['href'][:len(prefix)] == prefix and not link.text in exclude:
                    master_urls.append((link['href'], link.text))


    # Save the collected URLs in a text file
    with open("master_urls.txt", "a") as file:
        for url in master_urls:
            file.write(url[0] + "\n")

#### 1.2. Crawl master's degree pages

In [148]:
flag = False

if flag:
    # Open the file and read its content
    with open("master_urls.txt", "r") as file:
        master_urls = [line.strip() for line in file.readlines()]

    # Create a directory to store HTML pages
    output_root_directory = "html_pages"
    os.makedirs(output_root_directory, exist_ok=True)  # create the directory if it doen't exist

    # Read 15 URLs at a time and create HTML pages
    subset_size = 15
    for i in range(0, len(master_urls), subset_size):
        subset = master_urls[i:i + subset_size] # extract 15 more urls

        # Create a subfolder for each page
        output_directory = os.path.join(output_root_directory, f"page_{i // subset_size + 1}")
        os.makedirs(output_directory, exist_ok=True)

        # Create an HTML page for each URL in the subset
        for url in subset:
            prefix = 'https://www.findamasters.com/'
            response = requests.get(prefix + url)  # sends a GET request 

            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                page_content = soup.prettify()  # extract the content of the page          
                master_name = url.split("/")[-2]  # extract the name of the master from the URL 

                # Check if the file already exists, and append a number if necessary
                page_filename = f"{output_directory}/{master_name}.html"
                counter = 1
                while os.path.exists(page_filename):
                    page_filename = f"{output_directory}/{master_name}({counter}).html"
                    counter += 1

                # Save the content in a HTML file
                with open(page_filename, "w", encoding="utf-8") as file:
                    file.write(page_content)
            time.sleep(1.5)

        print(f"page {i // subset_size + 1} completed")
   

#### 1.3 Parse downloaded pages

In [149]:
##MY IDEA --> I have to create a function that retrieve the required information from a html page. in order to do so I exploit the structure of the file. 
#I use "try: except:" to avoid errors in the case that some information is missing

def extract_msc_page(msc_page_url):


    contents ={}
    # Parse HTML content
    page_soup = BeautifulSoup(msc_page_url, 'html.parser')

    #url
    try:
        canonical_link = page_soup.find('link', {'rel': 'canonical'})
        contents['url'] = canonical_link.get('href')
    except AttributeError:
        contents['url'] = ""


    # Course name
    try:
        page_links = page_soup.find('h1', {'class': 'text-white course-header__course-title'})
        name = page_links.get_text()
        contents["courseName"] = name.strip()
    except AttributeError:
        contents['courseName'] = ""

    # University name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__institution'})
        contents['universityName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['universityName'] = ""

    # Faculty name
    try:
        page_links = page_soup.find_all('a', {'class': 'course-header__department'})
        contents['facultyName'] = page_links[0].contents[0].strip()
    except (AttributeError, IndexError):
        contents['facultyName'] = ""

    # Full time
    try:
        page_links = page_soup.find('a', {'class': 'inheritFont concealLink text-decoration-none text-gray-600'})
        time = page_links.get_text().strip()
        contents['isItFullTime'] = time
    except AttributeError:
        contents['isItFullTime'] = ""

    # Description
    try:
        page_links = page_soup.find('div', {'id': 'Snippet'})
        description = page_links.get_text().strip()
        contents["description"] = description
    except AttributeError:
        contents['description'] = ""

    # Starting date
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__start-date py-2 pr-md-3 text-nowrap d-block d-md-inline-block'})
        starting = page_links.get_text().strip()
        contents["startDate"] = starting
    except AttributeError:
        contents['startDate'] = ""

    # Fees
    try:
        page_links = page_soup.find('div', {'class': 'course-sections course-sections__fees tight col-xs-24'})
        fees = page_links.get_text().strip()
        contents["fees"] = fees
    except AttributeError:
        contents['fees'] = ""

    # Modality
    try:
        page_links = page_soup.find('a', {'title': 'View all MSc courses'})
        modality = page_links.get_text().strip()
        contents["modality"] = modality
    except AttributeError:
        contents['modality'] = ""

    # Duration
    try:
        page_links = page_soup.find('span', {'class': 'key-info__content key-info__duration py-2 pr-md-3 d-block d-md-inline-block'})
        duration = page_links.get_text().strip()
        contents["duration"] = duration
    except AttributeError:
        contents['duration'] = ""

    # City
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__city'})
        city = page_links.get_text().strip()
        contents["city"] = city
    except AttributeError:
        contents['city'] = ""

    # Country
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__country'})
        country = page_links.get_text().strip()
        contents["country"] = country
    except AttributeError:
        contents['country'] = ""

    # Administration
    try:
        page_links = page_soup.find('a', {'class': 'card-badge text-wrap text-left badge badge-gray-200 p-2 m-1 font-weight-light course-data course-data__on-campus'})
        administration = page_links.get_text().strip()
        contents["administration"] = administration
    except AttributeError:
        contents["administration"] = ""

    
    return contents


In [150]:
## MY IDEA --> I apply the function above for all the files. I am applying it for all files in a folder for all folders contained into a folder

flag = False

if flag:
    folder_path = 'html_pages/'

    result_df = pd.DataFrame()
    # Walk through the directory and its subdirectories
    for foldername, subfolders, filenames in os.walk(folder_path):
        # Iterate through all the files in the current subdirectory
        for filename in filenames:
            # Construct the file path
            path = os.path.join(foldername, filename)
        
            # Check if the file exists before attempting to open it
            if os.path.exists(path):
                # Print the file path
                print(path)
            
                # Open the file in read mode with UTF-8 encoding
                with open(path, 'r', encoding='utf-8') as file:
                    # Read the HTML content of the file
                    html_content = file.read()
                
                    # Call the function to extract information from the HTML content
                    result_dict = extract_msc_page(html_content)
                    result_df = result_df.append(result_dict, ignore_index=True)


            else:
                # Print a message if the file is not found
                print(f"File not found: {path}")


    #clean an imperfection in the fees section
    result_df['fees'] = result_df['fees'].str.replace('Fees', '') 

    #save it into a file to store it. so that I do not have to run it again
    result_df.to_json('html_pages.json', orient='records', lines=True)


In [151]:
#create the tsv files 

flag = False

if flag:
    data = pd.read_json("html_pages.json", lines=True)

    output_directory = "tsv_files/"
    os.makedirs(output_directory, exist_ok=True)

    # Iterate through each row and create a TSV file
    for index, row in data.iterrows():
        file_name = f"course_{index}.tsv"
        file_path = os.path.join(output_directory, file_name)

        # Extract relevant columns and write to the TSV file
        selected_columns = ['courseName', 'universityName', 'facultyName', 'isItFullTime', 'description', 'startDate',
                        'fees','modality','duration','city','country','administration', 'url']
        selected_data = row[selected_columns]
        selected_data.to_csv(file_path, sep='\t', index=False, header=False)

    print("TSV files created successfully.")

### 2. Search Engine

##### 2.0.0) Preprocessing the text

In [6]:
dataset = pd.read_json("html_pages.json", lines=True)  # read the dataset from the created json file
dataset = dataset[dataset.description != '']  # filter rows where the 'description' is empty

# STEMMING
stemmer = PorterStemmer()  # create an instance of Porter Stemmer
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in row.split(' ')])  # reduce words of description column to their root form and create a new column to store the result

# STOPWORDS
nltk.download('stopwords') 
list_stopwords = stopwords.words('english')  # retrieves the English stopwords from the nltk stopwords dataset
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in row.split(' ') if not word in list_stopwords])  # now 'descr_clean' column contains lists of cleaned and stemmed words 

# PUNCTUATION
nltk.download('punkt')
dataset['preprocessed_description'] = dataset.description.apply(lambda row: [stemmer.stem(word) for word in nltk.word_tokenize(row) if not word in list_stopwords and word.isalnum()]) # now 'descr_clean' column contains lists of cleaned and stemmed words without punctuation

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mario\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


#### 2.0.1) Preprocessing the fees column

In [ ]:
import re
from forex_python.converter import CurrencyRates

pattern1 = r'([$£¥€]|USD|GBP|EUR|euro|euros|JPY|HK$|HK|HKD$|HKD|ISK|SEK)\s*([\d,]+(?:\.\d+)?)'  # regex to obtain matches as "symbol number"
pattern2 = r'([\d,]+(?:\.\d+)?)\s*([$£¥€]|USD|GBP|EUR|euro|euros|JPY|HK$|HK|HKD$|HKD|ISK|SEK)'  # regex to obtain matches as "number symbol"

# Create a CurrencyRates instance
currency_rates = CurrencyRates()

def extract_and_convert(row, common_currency="USD"):

    matches1 = re.finditer(pattern1, row)
    matches2 = re.finditer(pattern2, row)

    max_value = 0
    max_currency = None
    max_match = None

    
    for match in matches1:
        currency_symbol = match.group(1)
        numeric_part = match.group(2)
        
        
        # Try to convert numeric part to float
        try:
            numeric_value = float(numeric_part.replace(',', '').replace('.', ''))
        except ValueError:
            continue
        
        if numeric_value > max_value:  # update max value if necessary
            max_value = numeric_value
            max_currency = currency_symbol
            max_match = match
    
    for match in matches2:
        currency_symbol = match.group(2)
        numeric_part = match.group(1)
        
        
        # Try to convert numeric part to float
        try:
            numeric_value = float(numeric_part.replace(',', '').replace('.', '')) # remove all the number separators to do the cast
        except ValueError:
            continue
        
        if numeric_value > max_value: # update max value if necessary
            max_value = numeric_value
            max_currency = currency_symbol
            max_match = match
    
    if max_match:
        # Convert the price to the common currency
        converted_price = convert_currency(max_currency, max_value, common_currency)
        return converted_price
    else:
        return None


# Define a function to convert currency
def convert_currency(currency_symbol, amount, target_currency):
    try:
        # Map currency symbols to ISO codes
        currency_code = symbol_to_code(currency_symbol)
        
        if currency_code is not None:
            # Use the forex-python library to get the exchange rate and perform the conversion
            exchange_rate = currency_rates.get_rate(currency_code, target_currency)
            converted_amount = round(amount * exchange_rate, 2)
            print(f"Previous price: {amount}{currency_symbol}")
            print(f"Converted Price: {converted_amount:.2f} USD")
            return converted_amount
        else:
            print(f"Unsupported currency symbol: {currency_symbol}")
            return None
    except Exception as e:
        print(f"Error converting currency: {e}")
        return None

# Define a function to map currency symbols to ISO codes
def symbol_to_code(symbol):
    symbol_map = {
        '$': 'USD',
        '£': 'GBP',
        '¥': 'JPY',
        '€': 'EUR',
        'USD': 'USD',
        'GBP': 'GBP',
        'EUR': 'EUR',
        'euro': 'EUR',
        'euros': 'EUR',
        'JPY': 'JPY',
        'HKD$': 'HKD',
        'HKD': 'HKD',
        'HK$': 'HKD',
        'HK': 'HKD',
        'ISK': 'ISK',
        'SEK': 'SEK'
    }
    return symbol_map.get(symbol, None)


# Apply the function to the 'fees' column and create a new 'fees(USD)' column
dataset['fees(USD)'] = dataset['fees'].apply(lambda x: extract_and_convert(x, common_currency='USD'))
dataset


### 2.1. Conjunctive query

#### 2.1.1) Create your index!

In [ ]:
#step 1:
#Create a file named vocabulary, in the format you prefer, that maps each word to an integer (term_id).

vocabulary = set()
dataset.preprocessed_description.apply(lambda row: [vocabulary.add(word) for word in row])

In [ ]:
vocabulary_alt = Counter(reduce(lambda x, y: x + y, dataset.preprocessed_description.values)).keys()

index = {}
unique_id = 1
for word in list(vocabulary_alt):
  index[unique_id] = word
  unique_id += 1

# Create a DataFrame from the vocabulary
vocabulary_df_alt = pd.DataFrame(list(index.items()), columns=['term_id', 'word'])

# Save the vocabulary DataFrame to a CSV file
vocabulary_df_alt.to_csv('vocabulary_alt.csv', index=False)


terms = pd.DataFrame(data=list(vocabulary_alt), columns=['term'])

#### 2.1.2) Execute the query


In [ ]:
personalized_query = str(input())

#personalized_query = "3d group"

# Split the personalized query into individual words
query_words = personalized_query.split()

# Your modified code to get the indices of documents containing all words in the personalized query
selected_indices = list(dataset.loc[dataset.preprocessed_description.apply(lambda row: all(word in row for word in query_words))].index)

# Extract information for each selected document
selected_documents = dataset.loc[selected_indices, ['courseName', 'universityName', 'description', 'url']]

# Print the information for each selected document
print(selected_documents)

### 2.2) Conjunctive query & Ranking score

#### 2.2.1) Inverted index

In [ ]:
tfidf = TfidfVectorizer(input = "content", lowercase= False, tokenizer = lambda text: text, max_df=0.5, min_df = 2)

results = tfidf.fit_transform(dataset.preprocessed_description)
result_dense = results.todense()
pd.DataFrame(result_dense.tolist(), index=dataset.index, columns=tfidf.get_feature_names_out())


tfidf_data = pd.DataFrame(result_dense.tolist(), index=dataset.index, columns=tfidf.get_feature_names_out())
cosine_sim = cosine_similarity(tfidf_data)

In [ ]:
def search_engine(query, k=5):
    # Tokenize and preprocess the query
    query_tfidf = tfidf.transform([query])

    # Compute cosine similarity between the query and all documents
    query_similarity = cosine_similarity(query_tfidf, tfidf_data)

    # Create a heap to maintain the top-k documents
    heap = []

    # Iterate through the documents and update the heap
    for i, sim_score in enumerate(query_similarity[0]):
        heapq.heappush(heap, (sim_score, i))

    # Retrieve the top-k documents from the heap
    top_k_documents = []
    while heap and len(top_k_documents) < k:
        sim_score, index = heapq.heappop(heap)
        if sim_score > 0:
            document_info = {
                'courseName': dataset.loc[index, 'courseName'],
                'universityName': dataset.loc[index, 'universityName'],
                'description': dataset.loc[index, 'description'],
                'url': dataset.loc[index, 'url'],
                'Similarity Score': sim_score
            }
            top_k_documents.append(document_info)

    result_df = pd.DataFrame(top_k_documents)

    return result_df


##### 2.2.2) Execute the query

In [ ]:
#USING IT BY CHANGING THE CODE
query = "advanced knowledge"
top_k_results = search_engine(query, k=5)


#USING IT BY DOING AN INPUT.
personalized_query = str(input())
top_k_results = search_engine(personalized_query, k=5)

print(top_k_results)